# Exemple of pipeline PCA-GP-PCA type

In [1]:
from sklearn.datasets import make_regression

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

## Generate some synthetic regression data

In [2]:
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, n_targets=11)

## PCA-GP-PCA as an sklearn pipeline

### 1. Define the PCA for the shape embedding

In this example we only apply PCA to the first 8 columns

The last two columns are unchanged

In [3]:
feats_to_reduce = list(range(8))
preprocessor = ColumnTransformer(
    transformers=[
        (
            "pca",
            PCA(n_components=8),
            feats_to_reduce,
        ),
    ],
    remainder="passthrough",
)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pca', PCA(n_components=8),
                                 [0, 1, 2, 3, 4, 5, 6, 7])])

### 2. Define the output scaler for the output fields (MinMaxScaler + PCA)

In [4]:
postprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=9)),
    ]
)
postprocessor

Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=9))])

### 3. Define the regressor

Y = GP(transformer(X)) where transformer(X) = postprocessor(X)

In [5]:
regressor = TransformedTargetRegressor(
    regressor=GaussianProcessRegressor(
        n_restarts_optimizer=3,
    ),
    check_inverse=False,
    transformer=postprocessor,
)
regressor

TransformedTargetRegressor(check_inverse=False,
                           regressor=GaussianProcessRegressor(n_restarts_optimizer=3),
                           transformer=Pipeline(steps=[('scaler',
                                                        MinMaxScaler()),
                                                       ('pca',
                                                        PCA(n_components=9))]))

### 4. Combine to make the pipeline

In [6]:
model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler()),
        ("regressor", regressor),
    ]
)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pca', PCA(n_components=8),
                                                  [0, 1, 2, 3, 4, 5, 6, 7])])),
                ('scaler', StandardScaler()),
                ('regressor',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=GaussianProcessRegressor(n_restarts_optimizer=3),
                                            transformer=Pipeline(steps=[('scaler',
                                                                         MinMaxScaler()),
                                                                        ('pca',
                                                                         PCA(n_components=9))])))])

## Fit the model

In [7]:
model.fit(X, y)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pca', PCA(n_components=8),
                                                  [0, 1, 2, 3, 4, 5, 6, 7])])),
                ('scaler', StandardScaler()),
                ('regressor',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=GaussianProcessRegressor(n_restarts_optimizer=3),
                                            transformer=Pipeline(steps=[('scaler',
                                                                         MinMaxScaler()),
                                                                        ('pca',
                                                                         PCA(n_components=9))])))])

## Predict on the training data

In [8]:
y_pred = model.predict(X)

## Other way to define the pipeline

### 1. Define the regressor

In [9]:
regressor = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler()),
        ("regressor", GaussianProcessRegressor(n_restarts_optimizer=3)),
    ]
)
regressor

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pca', PCA(n_components=8),
                                                  [0, 1, 2, 3, 4, 5, 6, 7])])),
                ('scaler', StandardScaler()),
                ('regressor',
                 GaussianProcessRegressor(n_restarts_optimizer=3))])

### 2. Combine to make the pipeline

In [10]:
model = TransformedTargetRegressor(
    regressor=regressor,
    check_inverse=False,
    transformer=postprocessor,
)
model

TransformedTargetRegressor(check_inverse=False,
                           regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('pca',
                                                                                       PCA(n_components=8),
                                                                                       [0,
                                                                                        1,
                                                                                        2,
                                                                                        3,
                                                                                        4,
                                                                                        5,
                                                                                        6,
                                                                                        7])])),
                                                     ('scaler',
                                                      StandardScaler()),
                                                     ('regressor',
                                                      GaussianProcessRegressor(n_restarts_optimizer=3))]),
                           transformer=Pipeline(steps=[('scaler',
                                                        MinMaxScaler()),
                                                       ('pca',
                                                        PCA(n_components=9))]))

In [11]:
model.fit(X,y)

TransformedTargetRegressor(check_inverse=False,
                           regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('pca',
                                                                                       PCA(n_components=8),
                                                                                       [0,
                                                                                        1,
                                                                                        2,
                                                                                        3,
                                                                                        4,
                                                                                        5,
                                                                                        6,
                                                                                        7])])),
                                                     ('scaler',
                                                      StandardScaler()),
                                                     ('regressor',
                                                      GaussianProcessRegressor(n_restarts_optimizer=3))]),
                           transformer=Pipeline(steps=[('scaler',
                                                        MinMaxScaler()),
                                                       ('pca',
                                                        PCA(n_components=9))]))